## 1. A simulation exercise in regularization framework: sparsity in population coefficients

In [2]:
using Pkg
using CSV
using Distributions
using DataFrames
using Dates
using Plots
using Random
using LinearAlgebra
using LaTeXStrings
using Lasso
using GLM
using MLBase
using Statistics
using GLMNet

In [ ]:
function gen_data(n::Int, p::Int, regime::String="sparse")

    # constants chosen to get R^2 of approximately .80
    if regime == "sparse"
        beta = ((1 ./ [1:p;]) .^ 2) * 7
    elseif regime == "dense"
        beta = sort(rand(Uniform(0.9, 1), p), rev=true)
    end

    function true_fn(x)
        return x * beta
    end

    X = rand(Uniform(-0.5, 0.5), n, p)
    gX = true_fn(X)
    y = gX .+ randn(n)
    Xpop = rand(Uniform(-0.5, 0.5), 100000, p)  # almost population limit
    gXpop = true_fn(Xpop)
    ypop = gXpop .+ randn(100000)
    return X, y, gX, Xpop, ypop, gXpop, beta
end

### Approximate sparse coefficients

We are going to simulate a data generating process which coefficients vector is sparse. A set of coefficients is said to be approximate sparse if it satisfies the following property:


\begin{equation*}

\beta_j \propto \frac{1}{j^2}, \quad \forall j=1,2,\dots,p

\end{equation*}

In other words, the predictive power relies in a small subset of the coefficients vector (the larger ones).

In [6]:
n = 100
p = 400
X, y, gX, Xpop, ypop, gXpop, betas = gen_data(n, p, "sparse");

We can plot the set of (sorted) coefficients' magnitudes to visualize their behavior

In [ ]:
plot(1:length(betas),abs.(betas),seriestype=:path,color=:blue, labels = false,
    marker=:circle,markersize=4,xlabel=L"\beta",ylabel="Magnitude",
    title=L"\beta \ Magnitude")


As you may already notice, we are woking in a high dimensional setting ($p>n$). Let's test the dimensional reduction capabilities of the regularization methods. We would like to select only features which provide the most predictive power.

Remember that the general regularization problem (in p-norm) is the following:

\begin{equation*}

\min_{\beta_j \in \beta} \sum_{i=1}^{n}(Y_i - \beta^{\prime}X_i)^2 + \lambda\lVert \beta \lVert_p

\end{equation*}

where

\begin{equation*}

\lVert \beta \lVert_p = \left(\sum_{\beta_j \in \beta}\left|{\beta_{j}}^{p}\right|\right) ^{1/p}

\end{equation*}

When $p=1$, it is a Lasso (or L1) regularization . When $p=2$, it is a Ridge (or L2) regularization. We will explore the latter it in the next sessions

So, we fit a Lasso regularization. To tune the penalization parameter $\lambda$, we perform a cross-validation Lasso regression with $k=5$ folds.

In [61]:
model_l1 = glmnetcv(X, y, alpha=1.0, intercept=false, nfolds=5)
betas_l1 = model_l1.path.betas[:,argmin(model_l1.meanloss)];

Let's plot the estimated coefficients vs the population parameters

In [ ]:
plot(1:19, abs.(betas[1:19]), label="Population", color=:blue, marker=:circle, markersize=5)
plot!(1:19, abs.(betas_l1[1:19]), label="Lasso", color=:green, marker=:circle, markersize=5)
title!(L"First 20 larger $\beta$s")
xlabel!(L"$\beta$")
ylabel!("Magnitude")

The main conclusion of this exercise is that Lasso regularization fits scenearios of approximate sparse coefficients because it select a small subset with high predictive power. The linear predictor provided by Lasso may not be the BLP, but works well under high dimensionality.

## 2. A Simple Case Study using Wage Data from 2015

### Data

We consider data from the U.S. March Supplement of the Current Population Survey (CPS) in 2015.
The preproccessed sample consists of $5150$ never-married individuals.

In [3]:
using HTTP

In [ ]:
file = "https://raw.githubusercontent.com/CausalAIBook/MetricsMLNotebooks/main/data/wage2015_subsample_inference.csv"
data = CSV.read(download(file), DataFrame,
                types = Dict(:occ2 => String, :ind2 => String))
                

In [ ]:
y = data.lwage
Z = select(data, Not([:wage, :lwage]))
column_names = names(Z)

The following figure shows the weekly wage distribution from the US survey data.

In [ ]:
histogram(data.wage, bins=0:20:350, xlabel="hourly wage", ylabel="Frequency",
          title="Empirical wage distribution from the US survey data", label =false, ylim=(0, 3000))

Wages show a high degree of skewness. Hence, wages are transformed in almost all studies by the logarithm.

Due to the skewness of the data, we are considering log wages which leads to the following regression model

$$\log(\operatorname{wage}) = g(Z) + \epsilon.$$

In this notebook, we will evaluate *linear* prediction rules. In later notebooks, we will also utilize nonlinear prediction methods. In linear models, we estimate the prediction rule of the form

$$\hat g(Z) = \hat \beta'X.$$

Again, we generate $X$ in three ways:

1. Basic Model:   $X$ consists of a set of raw regressors (e.g. gender, experience, education indicators, regional indicators).


2. Flexible Model:  $X$ consists of all raw regressors from the basic model plus occupation and industry indicators, transformations (e.g., $\operatorname{exp}^2$ and $\operatorname{exp}^3$) and additional two-way interactions.

To evaluate the out-of-sample performance, we split the data first.

The basic model consists of $51$ regressors, and the flexible model has $246$ regressors. Let us fit our models to the training sample using the two different model specifications. We are starting by running a simple ols regression and computing the $R^2$ on the test sample.

### Low dimensional specification (basic)

In [123]:
basic_formula = @formula(lwage ~1+ sex + exp1 + shs + hsg+ scl + clg + mw + so + we + occ2+ ind2)
Z_base = modelmatrix(basic_formula, data);

In [124]:
train_sample = rand(Float64, size(data)[1]) .< 0.75
test_sample = .!(train_sample)
y_train, y_test = y[train_sample], y[test_sample]
X_train, X_test = Z_base[train_sample, :], Z_base[test_sample, :];

([1.0 1.0 … 0.0 0.0; 1.0 0.0 … 0.0 1.0; … ; 1.0 0.0 … 0.0 0.0; 1.0 0.0 … 0.0 0.0], [1.0 1.0 … 0.0 0.0; 1.0 1.0 … 0.0 0.0; … ; 1.0 1.0 … 0.0 0.0; 1.0 0.0 … 0.0 0.0])

In [125]:
lr_base = lm(X_train, y_train);

Let's calculate R-squared on the test set

In [ ]:
basic_mse_testing = mean((predict(lr_base, X_test) - y_test) .^ 2)
basic_r2_testing = 1 - basic_mse_testing / var(y_test)

### High-dimensional specification (flexible)

We repeat the same procedure for the flexible model.

In [137]:
flex_formula = @formula(lwage ~1 + sex + (exp1+exp2+exp3+exp4)*(shs+hsg+scl+clg+occ2+ind2+mw+so+we))
Z_flex = modelmatrix(flex_formula, data);

In [138]:
X_train, X_test = Z_flex[train_sample, :], Z_flex[test_sample, :];

In [139]:
lr_flex = lm(X_train, y_train);

In [140]:
basic_mse_testing = mean((predict(lr_flex, X_test) - y_test) .^ 2)
basic_r2_testing = 1 - basic_mse_testing / var(y_test)

0.2808300171229896

### Penalized regression: Lasso (flexible model)

We observe that ols regression works better for the basic model with smaller $p/n$ ratio. We proceed by running a Lasso regression for the flexible model, tuned via cross-validation.

To properly penalize the coefficients, we must standarize the data, so each regressor is symmetrically penalized 

As in R, Julia's GLMnet standarizes variables automatically

In [158]:
lasso_flex = glmnetcv(X_train, y_train, alpha=1.0, intercept=false, nfolds=5)
betas_flex = lasso_flex.path.betas[:,argmin(lasso_flex.meanloss)];
prediction = X_test * betas_flex
basic_mse_testing = mean((prediction - y_test) .^ 2)
basic_r2_testing = 1 - basic_mse_testing / var(y_test)

-18.507151527143755